# 1. Import the necessary libraries for you to be able to query 4square API

In [6]:
import requests
import json
import pandas as pd
import getpass
import requests
import os

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [2]:
# Get Foursquare API key 
foursquare_token = getpass.getpass("Enter your Foursquare API token: ")

# Ironhack Madrid Location 
ironhack_location = {'lat': 40.3925, 'lng': -3.6983} 

# Foursquare API URL
url = 'https://api.foursquare.com/v3/places/search'

# Parameters for the API request
params = {
    'query': 'coffee',
    'll': f"{ironhack_location['lat']},{ironhack_location['lng']}",
    'limit': 10,  
    'radius': 1000  
}

# Headers 
headers = {
    'Authorization': foursquare_token
}

# Send the request to the Foursquare API
response = requests.get(url, params=params, headers=headers)

# Check the status of the request
if response.status_code == 200:
    # Parse the JSON response
    coffee_places = response.json()
    
    # Display the names and addresses of the top 10 coffee places
    for idx, place in enumerate(coffee_places['results']):
        name = place['name']
        address = place['location']['formatted_address']
        print(f"{idx + 1}. {name} - {address}")
else:
    print(f"Error: {response.status_code} - {response.text}")

1. Cantina Matadero - P. de la Chopera, 14, 28045 Madrid Comunidad de Madrid
2. La Francachela - Paseo de la Chopera, 14, 28045 Madrid Comunidad de Madrid
3. Dot Café Bar - Calle de Eugenio Sellés, 6, 28045 Madrid Comunidad de Madrid
4. Cafetería Churrería Legazpi - Plaza Legazpi, 3, 28045 Madrid Comunidad de Madrid
5. Rodilla - Calle de Antonio López, 109, 28026 Madrid Comunidad de Madrid
6. El Gallo de Oro - Antonio López, 113, 28026 Madrid Comunidad de Madrid
7. Salon de Te al Yabal - Cáceres, 52 (P. Santa María de la Cabeza), 28045 Madrid Comunidad de Madrid
8. BBIG Delice Café - Calle Juan de Vera, 11, 28045 Madrid Comunidad de Madrid
9. Heladeria Venezzia - Paseo de las Delicias, 102 (C. Divino Vallés), 28045 Madrid Comunidad de Madrid
10. Viena Capellanes - P.º Molino, 6 (C. Frigiliana), 28045 Madrid Comunidad de Madrid


# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [3]:
# Get Foursquare API key securely
foursquare_token = getpass.getpass("Enter your Foursquare API token: ")

# Foursquare API URL
url = 'https://api.foursquare.com/v3/places/search'

# Parameters for the API request
params = {
    'query': 'coffee',
    'near': 'Ironhack Madrid',  
    'limit': 5,  
}

# Headers 
headers = {
    'Authorization': foursquare_token
}

# Send the request to the Foursquare API
response = requests.get(url, params=params, headers=headers)

# Check the status of the request
if response.status_code == 200:
    # Parse the JSON response
    coffee_places = response.json()
    
    # Display the names and addresses of the top 5 closest coffee places
    for idx, place in enumerate(coffee_places['results']):
        name = place['name']
        address = place['location']['formatted_address']
        print(f"{idx + 1}. {name} - {address}")
else:
    print(f"Error: {response.status_code} - {response.text}")

1. Bucólico Café - Calle Barbieri, 4, 28004 Madrid Comunidad de Madrid
2. La Rollerie - Carrera San Jerónimo, 26, 28014 Madrid Comunidad de Madrid
3. Chocolatería San Ginés - Pasadizo de San Ginés, 5 (C. Arenal), 28013 Madrid Comunidad de Madrid
4. El Jardín de Salvador Bachiller - Calle Montera, 37, 28013 Madrid Comunidad de Madrid
5. Pum Pum Bakery - Calle de la Esgrima, 1, 28012 Madrid Comunidad de Madrid


# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [4]:
def foursquare_places(venue, coordinates, limit=10):
    """
    Finds places of a specified type near a given set of coordinates.
    
    Parameters:
    - venue (str): The type of place to search for (e.g., "Starbucks" or "restaurant").
    - coordinates (dict): A dictionary containing 'latitude' and 'longitude'.
    - limit (int): Number of results to return (default is 10).
    
    Returns:
    - List of dictionaries with name, latitude, and longitude of each place.
    """
    # Foursquare API URL
    url = 'https://api.foursquare.com/v3/places/search'
    
    # Parameters for the API request
    params = {
        'query': venue,
        'll': f"{coordinates['latitude']},{coordinates['longitude']}",  # Coordinates for location
        'limit': limit  # Specify the maximum number of results
    }
    
    # Headers
    headers = {
        'Authorization': foursquare_token
    }
    
    # Send the request to the Foursquare API
    response = requests.get(url, params=params, headers=headers)
    
    # Check the status of the request
    if response.status_code == 200:
        # Parse the JSON response
        places = response.json().get("results", [])
        
        # Extract name, latitude, and longitude
        results = []
        for place in places:
            name = place.get('name', 'N/A')
            lat = place['geocodes']['main']['latitude']
            lon = place['geocodes']['main']['longitude']
            results.append({
                "Name": name,
                "Latitude": lat,
                "Longitude": lon
            })
        return results
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return []

# Example usage:
coordinates = {'latitude': 40.3925, 'longitude': -3.6983}  
venue = "Starbucks"  
results = foursquare_places(venue, coordinates, limit=5)

# Display results
for place in results:
    print(f"Name: {place['Name']}, Latitude: {place['Latitude']}, Longitude: {place['Longitude']}")

Name: Starbucks, Latitude: 40.391357, Longitude: -3.701159
Name: Starbucks, Latitude: 40.408901, Longitude: -3.693309
Name: Starbucks, Latitude: 40.415289, Longitude: -3.695056
Name: Starbucks, Latitude: 40.420652, Longitude: -3.698997
Name: Starbucks, Latitude: 40.417206, Longitude: -3.706114


# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [7]:
def save_to_json(data, filename="data/coffee_shops.json"):
    """
    Saves data to a JSON file in the specified filename.
    
    Parameters:
    - data: Data to be saved (list of dictionaries).
    - filename: Path to the JSON file (default is 'data/coffee_shops.json').
    """
    # Ensure the data folder exists
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    
    # Write data to a JSON file
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)
    print(f"Data saved to {filename}")

# Example usage:
coordinates = {'latitude': 40.3925, 'longitude': -3.6983}  
venue = "coffee"  
results = foursquare_places(venue, coordinates, limit=5)

# Save the results to JSON
if results:
    save_to_json(results, "data/coffee_shops.json")
else:
    print("No results to save.")

Data saved to data/coffee_shops.json


# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [ ]:
# your code here